<a href="https://colab.research.google.com/github/marcory-hub/swift-yolo/blob/main/Swift_YOLO_192_gdrive_eval_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a Swift-YOLO 192×192 model with a dataset from Google Drive
This notebook trains a **Swift-YOLO** (tiny, 192×192) object-detection model using [ModelAssistant](https://github.com/Seeed-Studio/ModelAssistant) (SSCMA). Your dataset lives in Google Drive; the notebook copies it into Colab and runs training.

Last accessed: 2026-02-05

**Category:** Object Detection

**Algorithm:** [Swift-YOLO](configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py)

**Dataset:** Custom dataset from Google Drive (requires Gmail account)

---

## Setup instructions

Do these steps **before** running the notebook.

### Step 1: Enable GPU

1. In Colab: **Runtime** → **Change runtime type**
2. Set **Hardware accelerator** to **GPU** (choose **A100** if available for faster training)
3. Click **Save**

**Expected environment** (check the first code cell output):
- GPU: T4 or higher (A100 preferred for long runs)
- Python 3.11.x

---

### Step 2: Prepare your dataset in COCO format

1. **Folder structure** — Use this layout (exact names). Each of `train/` and `valid/` must contain images and one `_annotations.coco.json`:

```
dataset
├── train
│   ├── _annotations.coco.json
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
└── valid
    ├── _annotations.coco.json
    ├── image3.jpg
    ├── image4.jpg
    └── ...
```

2. **Zip the dataset** — Create `dataset.zip` from the `dataset` folder. On macOS/Linux you can exclude metadata with:

   `zip -r dataset.zip dataset -i '*.jpg' '*.jpeg' '*.png' '*.json'`

3. **Upload to Google Drive** — Put `dataset.zip` in the root directory. Optionally put a custom pretrained checkpoint there too (e.g. `best_coco_bbox_mAP_epoch_100.pth`) if you want to fine-tune from your own model.

---

### Step 3: Set number of classes

⚠️ **Update the class count to match your dataset.**

- **Default in this notebook:** 4 classes
- **What to do:** In the training cell, set `num_classes` (and in the config override) to your actual number of classes (e.g. `num_classes=5` for 5 classes).

---

## Ready to run

Run the cells in order. The notebook will:

1. Mount Google Drive and check GPU/Python
2. Clone ModelAssistant and install dependencies
3. Download pretrained weights and copy your dataset from Drive
4. Train Swift-YOLO (default: 3 epochs for a quick test; use 100+ for real training)
5. Zip the output folder so you can download checkpoints and exports

## 1. Check GPU and Python version

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!free -h
!df -h

               total        used        free      shared  buff/cache   available
Mem:            50Gi       2.6Gi        44Gi       3.0Mi       4.3Gi        47Gi
Swap:             0B          0B          0B
Filesystem      Size  Used Avail Use% Mounted on
overlay         242G   42G  201G  17% /
tmpfs            64M     0   64M   0% /dev
shm              25G     0   25G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/tini
tmpfs            26G   76K   26G   1% /var/colab
/dev/sda1       242G   42G  201G  17% /kaggle/input
tmpfs            26G     0   26G   0% /proc/acpi
tmpfs            26G     0   26G   0% /sys/firmware
tmpfs            26G     0   26G   0% /proc/scsi
drive            15G  9.3G  5.8G  62% /content/drive


In [3]:
# Check if GPU is available
!nvidia-smi

# Check Python version
!python --version

Thu Feb  5 19:54:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Prerequisites: clone ModelAssistant and install SSCMA
Clone the [ModelAssistant](https://github.com/Seeed-Studio/ModelAssistant) repo and run the Colab setup script. You can ignore any non-fatal errors from the setup script.

In [11]:
!git config --global credential.helper store

In [13]:
!pip install ethos-u-vela
!git clone -b 2.0.0 https://github.com/Seeed-Studio/ModelAssistant.git
%cd ModelAssistant
!./scripts/setup_colab.sh

Cloning into 'ModelAssistant'...
remote: Enumerating objects: 15755, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 15755 (delta 7), reused 17 (delta 2), pack-reused 15719 (from 1)
Receiving objects: 100% (15755/15755), 26.61 MiB | 21.19 MiB/s, done.
Resolving deltas: 100% (9182/9182), done.
/content/ModelAssistant
Checking if CUDA available... OK
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 129.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 161.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━


## 3. Download pretrained weights

In [14]:
%mkdir -p Digital_Meter_Water_Swift-YOLO_192
!wget -c https://files.seeedstudio.com/sscma/model_zoo/detection/water_meter/yolov5_tiny_1xb16_300e_coco_sha1_e10d262518622edc50e0820b213581fc8d628e2b.pth -O Digital_Meter_Water_Swift-YOLO_192/pretrain.pth

--2026-02-05 20:13:20--  https://files.seeedstudio.com/sscma/model_zoo/detection/water_meter/yolov5_tiny_1xb16_300e_coco_sha1_e10d262518622edc50e0820b213581fc8d628e2b.pth
Resolving files.seeedstudio.com (files.seeedstudio.com)... 3.169.121.40, 3.169.121.91, 3.169.121.89, ...
Connecting to files.seeedstudio.com (files.seeedstudio.com)|3.169.121.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12454991 (12M) [application/octet-stream]
Saving to: ‘Digital_Meter_Water_Swift-YOLO_192/pretrain.pth’

Digital_Meter_Water 100%[===================>]  11.88M  10.3MB/s    in 1.2s    

2026-02-05 20:13:22 (10.3 MB/s) - ‘Digital_Meter_Water_Swift-YOLO_192/pretrain.pth’ saved [12454991/12454991]



## 4. Copy dataset from Google Drive

Copies `dataset.zip` from `/content/drive/MyDrive/` into Colab and unzips it under `Swift-YOLO_192/`. Ensure you created the `yolo` folder and put `dataset.zip` there (see Setup Step 2). Overwrites if files already excist.


In [15]:
!cp '/content/drive/MyDrive/dataset.zip' '/content/dataset.zip'
!unzip -o '/content/dataset.zip' -d '/content/ModelAssistant/Swift-YOLO_192/'

Streaminguitvoer ingekort tot de laatste 5000 regels.
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/amel2_0394_jpg.rf.f27129d4ec1972ff0844693fe7b6d4a8.jpg  
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/amel2_7078_jpg.rf.509fb2c1f75a073e37d46d40b2d7726c.jpg  
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/amel2_3524_jpg.rf.6f0f82aa1ecebf2da33d74ff7fec01a0.jpg  
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/vespsp2_1146_jpg.rf.c2bbb1aa1612626bb686e3e92b07fbbb.jpg  
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/vvel2_5736_jpg.rf.98849f024e5269716002504f3f3d00ec.jpg  
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/amel1_2400_jpg.rf.affd88cd3e9ede1a77662441d10c2c36.jpg  
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/vvel1_0574_jpg.rf.624b50eff3f60143e765ffc331c46c03.jpg  
  inflating: /content/ModelAssistant/Swift-YOLO_192/dataset/train/vespsp1_1365_jpg.

### Optional: use your own pretrained checkpoint
If you have a trained Swift-YOLO checkpoint in Drive (e.g. `best_coco_bbox_mAP_epoch_100.pth`), copy it to `Swift-YOLO_192/pretrain.pth` to fine-tune from it. Put the file in `/content/drive/MyDrive/` and adjust the filename below if needed.

In [16]:
!cp '/content/drive/MyDrive/best_coco_bbox_mAP_epoch_100.pth' 'Swift-YOLO_192/pretrain.pth'

## 5. Train Swift-YOLO with SSCMA

Training uses the config `swift_yolo_tiny_1xb16_300e_coco.py`. Key parameters (overridden with `--cfg-options`):

| Parameter | Description |
|-----------|-------------|
| `data_root` | Path to dataset (train/valid with COCO JSONs) |
| `epochs` | Number of training epochs (**3 here = quick test**; use 100+ for real training) |
| `batch` | Batch size |
| `height`, `width` | Input size (192×192 for Swift-YOLO) |
| `load_from` | Pretrained checkpoint path |
| `num_classes` | Number of classes in your dataset |

Change these in the cell below to match your dataset and goal.

In [ ]:
BASE = '/content/ModelAssistant'
%cd {BASE}

!sscma.train configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
--cfg-options  \
    work_dir={BASE}/Swift-YOLO_192 \
    num_classes=4 \
    epochs=300  \
    height=192 \
    width=192 \
    batch=256 \
    lr=0.16 \
    data_root={BASE}/Swift-YOLO_192/dataset/ \
    load_from={BASE}/Swift-YOLO_192/pretrain.pth

/content/ModelAssistant
Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0
Using automatically generated input shape (from config 'swift_yolo_tiny_1xb16_300e_coco.py'): [1, 3, 192, 192]
02/05 20:16:37 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 447159159
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a

In [ ]:
import os
with open('Swift-YOLO_192/last_checkpoint', 'r') as f:
	os.environ['CHECKPOINT_FILE_PATH'] = f.read()

In [ ]:
!sscma.export configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py $CHECKPOINT_FILE_PATH --cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=100  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}.pth \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=100  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_float32.tflite \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=100  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_int8.tflite \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=100  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

In [ ]:
%ls -lh Swift-YOLO_192/

## 6. Zip and download the Swift-YOLO_192 folder

Packs checkpoints and exported files from `Swift-YOLO_192/` into a zip and triggers download. You can then export to `int8_vela.tflite` for Grove Vision AI V2 (see Deploy below) or use the `.pth` checkpoint for further training.

In [ ]:
import os
import zipfile
from google.colab import files

# Define the paths
folder_path = '/content/ModelAssistant/Swift-YOLO_192'
zip_filename = 'Swift-YOLO_Models.zip'

# Create the zip file
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # List all items in the directory
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)

        # Only add to zip if it's a file (skips all subdirectories)
        if os.path.isfile(item_path):
            zipf.write(item_path, arcname=item)
            print(f"Added to zip: {item}")

# Automatically trigger the download to your computer
files.download(zip_filename)